# Obtaining energy from IonQ jobs

In this notebook, we show how to get $\langle \psi|H_{\text{XXZ}}|\rangle$ where $\psi=\hat U |0\rang$ from simulation results of $\hat U$.

In [1]:
# toy circuit in qiskit H: creates a Bell state
from qiskit import QuantumCircuit
import numpy as np
from utils import *
nqubits = 3
qc = QuantumCircuit(3)
qc.h(1)
qc.cx(1, 2)
# qc.measure_all()
qc.draw()

q_0: ──────────
     ┌───┐     
q_1: ┤ H ├──■──
     └───┘┌─┴─┐
q_2: ─────┤ X ├
          └───┘

In [3]:
# initialize ionq
from qiskit import QuantumCircuit
from qiskit_ionq import IonQProvider
from copy import deepcopy

my_api_key = "pOiUVlzriOoF2wX1kp3lIqid1OMhwXZ5"
provider = IonQProvider(my_api_key)
simulator_backend = provider.get_backend("ionq_simulator")

In [4]:
qc_XYZ = rotate_circuit_XYZ(qc)

In [11]:
for qc_m in qc_XYZ:
    print(qc_m.draw())

        ┌───┐           ░ ┌─┐      
   q_0: ┤ H ├───────────░─┤M├──────
        ├───┤     ┌───┐ ░ └╥┘┌─┐   
   q_1: ┤ H ├──■──┤ H ├─░──╫─┤M├───
        └───┘┌─┴─┐├───┤ ░  ║ └╥┘┌─┐
   q_2: ─────┤ X ├┤ H ├─░──╫──╫─┤M├
             └───┘└───┘ ░  ║  ║ └╥┘
meas: 3/═══════════════════╩══╩══╩═
                           0  1  2 
        ┌─────┐┌───┐             ░ ┌─┐      
   q_0: ┤ Sdg ├┤ H ├─────────────░─┤M├──────
        └┬───┬┘└───┘┌─────┐┌───┐ ░ └╥┘┌─┐   
   q_1: ─┤ H ├───■──┤ Sdg ├┤ H ├─░──╫─┤M├───
         └───┘ ┌─┴─┐├─────┤├───┤ ░  ║ └╥┘┌─┐
   q_2: ───────┤ X ├┤ Sdg ├┤ H ├─░──╫──╫─┤M├
               └───┘└─────┘└───┘ ░  ║  ║ └╥┘
meas: 3/════════════════════════════╩══╩══╩═
                                    0  1  2 
                   ░ ┌─┐      
   q_0: ───────────░─┤M├──────
        ┌───┐      ░ └╥┘┌─┐   
   q_1: ┤ H ├──■───░──╫─┤M├───
        └───┘┌─┴─┐ ░  ║ └╥┘┌─┐
   q_2: ─────┤ X ├─░──╫──╫─┤M├
             └───┘ ░  ║  ║ └╥┘
meas: 3/══════════════╩══╩══╩═
                      0

In [18]:
# observable XXZ:
xxz = xxz_hamiltonian(nqubits)
zz = xxz_hamiltonian(nqubits,delta=1,select='ZZ')
coefs = [1,1,0.5]

In [20]:
shots = 1000
keys_ls = []
frequencies_ls = []
observable_labels = ['XX', 'YY', 'ZZ']
xxz_expectation = 0
for i,qc_m in enumerate(qc_XYZ):
    job = simulator_backend.run(qc_m, shots=shots)
    result = job.result()
    # Get the counts from the result
    counts = result.get_counts()
    keys = [key.split()[0][::-1] for key in counts.keys()]
    sample_counts = list(counts.values())
    frequencies = np.array(sample_counts) / sum(sample_counts)
    ob = zz
    coef = coefs[i]
    expectation = sample_to_expectation(ob.to_matrix(), keys, frequencies)*coef
    xxz_expectation += expectation
    print(observable_labels[i], expectation)
    keys_ls.append(keys)
    frequencies_ls.append(frequencies)
    
print('Expectation energy:', xxz_expectation)

0.8720000000000001
-1.0
0.484


In [4]:
# Run the circuit on the IonQ simulator
qc_m = deepcopy(qc)
qc_m.measure_all()
shots = 1000
job = simulator_backend.run(qc_m, shots=shots)
result = job.result()

# Get the counts from the result
counts = result.get_counts()
print(counts)

{'000': 498, '110': 502}


In [9]:
keys = [key.split()[0][::-1] for key in counts.keys()]
sample_counts = list(counts.values())
frequencies = np.array(sample_counts) / sum(sample_counts)
print(keys)
print(sample_counts)
print(frequencies)

['000', '011']
[498, 502]
[0.498 0.502]


## Qiskit calculation

In [29]:
from qiskit_ibm_runtime import QiskitRuntimeService
 
# If you did not previously save your credentials, use the following line instead:
service = QiskitRuntimeService(channel="ibm_quantum", token="d7b55bcdcf222ad6aa9d85d01405645658facca7ed1cd3b304eabe6ded9bc5d90a5e4b783431c844e7a606e5206d1adbd0246b29c4d71392ae14b8b4d94e9e18")
# service = QiskitRuntimeService()
 
backend = service.least_busy(simulator=False, operational=True)

In [33]:
from qiskit.quantum_info import Statevector
statevector = Statevector.from_instruction(qc)
for ob in observables:
    expectation = np.real(np.dot(statevector.data.conj().T, np.dot(ob.to_matrix(), statevector.data)))
    print(expectation)

0.49999999999999994
0.9999999999999998
-0.9999999999999998
0.49999999999999994
